This notebook provides a possibility to generate sensitivity reports.For generating them we need different tools, which we first have to install in Colab. In particular, we use the GLPK-solver. Outside of Colab this can be done much easier. However, this is not part of the lecture and the tutorials.


In [1]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh && bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

!conda install -c conda-forge glpk -y

--2020-12-01 14:17:04--  https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 651745206 (622M) [application/x-sh]
Saving to: ‘Anaconda3-5.2.0-Linux-x86_64.sh.1’

Anaconda3-5.2.0-Lin 100%[===================>] 621.55M   196MB/s    in 3.3s    

2020-12-01 14:17:07 (189 MB/s) - ‘Anaconda3-5.2.0-Linux-x86_64.sh.1’ saved [651745206/651745206]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: blas-1.0-mkl ...
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: intel-openmp-2018.0.0-8 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libgfortran-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: bzip2-1

In [2]:
!pip install pulp
from pulp import *

distributed 1.21.8 requires msgpack, which is not installed.


In [3]:
# model
prob = LpProblem("VehicleManufacturing", LpMaximize)

# variables 
x = LpVariable.dicts("production", ['c', 't'], 0)
m = LpVariable.dicts("machines", [1], 0)

# objective function
prob += 300*x['c']+400*x['t']-50*m[1]

# constraints
prob += 0.8*x['c']+ x['t']-m[1] <= 0 ,'days_machine1'
prob += m[1] <= 98, 'numbers_machine1'
prob += 0.6*x['c']+ 0.7*x['t'] <= 73, 'days_machine2'
prob += 2*x['c']+ 3*x['t'] <= 260,'steel'
prob += x['c']>= 88,'min_cars'
prob += x['t'] >= 26,'min_trucks'

In [4]:
#%cd /home
prob.writeLP("VehicleManufacturing.lp")
!glpsol --lp VehicleManufacturing.lp --ranges "/dev/stdout"

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --lp VehicleManufacturing.lp --ranges /dev/stdout
Reading problem data from 'VehicleManufacturing.lp'...
6 rows, 3 columns, 10 non-zeros
11 lines were read
GLPK Simplex Optimizer, v4.65
6 rows, 3 columns, 10 non-zeros
Preprocessing...
3 rows, 3 columns, 7 non-zeros
Scaling...
 A: min|aij| =  6.000e-01  max|aij| =  3.000e+00  ratio =  5.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 3
      0: obj =   3.680000000e+04 inf =   9.640e+01 (1)
      1: obj =   3.198000000e+04 inf =   0.000e+00 (0)
*     2: obj =   3.254000000e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40424 bytes)
Write sensitivity analysis report to '/dev/stdout'...
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  OBJ = 32540 (MAXimum)

   No. Ro

In [5]:
# model
prob = LpProblem("ComputerProduction", LpMaximize)

# variables 
x = LpVariable.dicts("production", ['NP', 'LP', 'NV', 'LV'], 0)
l = LpVariable.dicts("labour", [1], 0)

# objective function
prob += 600*x['NP']+1000*x['LP']+800*x['NV']+1300*x['LV']-20*l[1]

# constraints
prob += 2*x['NP']+3*x['LP']+2*x['NV']+4*x['LV']-l[1] <= 0 ,'labour'
prob += l[1] <= 4000, 'purchased_labour'

prob += x['NV'] + x['NP'] <= 800,'production_NewYork'
prob += x['LV'] + x['LP'] <= 1000,'production_LosAngeles'

prob += x['NP'] + x['LP'] <= 900,'selling_limitPC'
prob += x['NV'] + x['LV'] <= 900,'selling_limitVAX'

In [6]:
%cd /home
prob.writeLP("ComputerProduction.lp")
!glpsol --lp ComputerProduction.lp --ranges "/dev/stdout"

/home
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --lp ComputerProduction.lp --ranges /dev/stdout
Reading problem data from 'ComputerProduction.lp'...
6 rows, 5 columns, 14 non-zeros
13 lines were read
GLPK Simplex Optimizer, v4.65
6 rows, 5 columns, 14 non-zeros
Preprocessing...
5 rows, 5 columns, 13 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  4.000e+00  ratio =  4.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 5
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (4)
*     7: obj =   1.360000000e+06 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.0 Mb (40448 bytes)
Write sensitivity analysis report to '/dev/stdout'...
GLPK 4.65 - SENSITIVITY ANALYSIS REPORT                                                                         Page   1

Problem:    
Objective:  OBJ = 1360000 (MAXimum)

   No. Row name     St      Activity         Slack   Lower 